In [22]:
__brand = "Fanuc"
__start_page = 461
_end_page = 600
_brand_url = "fanuc"

In [23]:
#step 1
import requests
from bs4 import BeautifulSoup
import csv

start_page = __start_page
end_page = _end_page

output_file = 'fanuc_link.csv'

all_links = []

for i in range(start_page, end_page + 1):
    url = f'https://www.yaotaishcnc.com/{_brand_url}/index_{i}.html'
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        links = soup.find_all('a', class_='prtu')

        for link in links:
            href = link.get('href')
            if href:
                all_links.append([href])

        print(f"Page {i} scraped successfully.")
    except Exception as e:
        print(f"Failed to scrape page {i}: {e}")


with open(output_file, 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['Link'])
    writer.writerows(all_links)

print(f"Saved {len(all_links)} links to {output_file}.")

Page 461 scraped successfully.
Page 462 scraped successfully.
Page 463 scraped successfully.
Page 464 scraped successfully.
Page 465 scraped successfully.
Page 466 scraped successfully.
Page 467 scraped successfully.
Page 468 scraped successfully.
Page 469 scraped successfully.
Page 470 scraped successfully.
Page 471 scraped successfully.
Page 472 scraped successfully.
Page 473 scraped successfully.
Page 474 scraped successfully.
Page 475 scraped successfully.
Page 476 scraped successfully.
Page 477 scraped successfully.
Page 478 scraped successfully.
Page 479 scraped successfully.
Page 480 scraped successfully.
Page 481 scraped successfully.
Page 482 scraped successfully.
Page 483 scraped successfully.
Page 484 scraped successfully.
Page 485 scraped successfully.
Page 486 scraped successfully.
Page 487 scraped successfully.
Page 488 scraped successfully.
Page 489 scraped successfully.
Page 490 scraped successfully.
Page 491 scraped successfully.
Page 492 scraped successfully.
Page 493

In [24]:
#Step 2
import pandas as pd

df = pd.read_csv("fanuc_link.csv")

df["Link"] = "https://www.yaotaishcnc.com/" + df["Link"]

df.to_csv("input_urls.csv", index=False)

In [25]:
#Step 3

import os
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup

INPUT_CSV = "input_urls.csv"
OUTPUT_DIR = "Output CSV"
OUTPUT_FILE = os.path.join(OUTPUT_DIR, "products.csv")
BASE_URL = "https://www.yaotaishcnc.com/"

os.makedirs(OUTPUT_DIR, exist_ok=True)

# Read URLs from CSV
url_df = pd.read_csv(INPUT_CSV)
scraped_data = []

# Scrape each URL
for index, row in url_df.iterrows():
    url = row['Link']
    print(f"Scraping: {url}")

    try:
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.content, "html.parser")

        # Extract Product Title
        title_tag = soup.select_one(".jianjie em")
        product_title = title_tag.get_text(strip=True) if title_tag else "N/A"

        # Extract Product Images with full URLs
        image_tags = soup.select(".item-pic img")
        image_urls = []
        for img in image_tags:
            src = img.get("src")
            if src:
                full_url = src if src.startswith("http") else BASE_URL + src.lstrip("/")
                image_urls.append(full_url)
        product_images = ", ".join(image_urls)

        # Extract and clean Description
        desc_tag = soup.select_one(".jiann.mp")
        raw_description = desc_tag.get_text(separator=" ", strip=True) if desc_tag else "N/A"

        # Format description into separate lines for known fields
        formatted_description = re.sub(
            r"(Brand:|Module number:|Condition:|Quality:|Warranty:|Shipment Terms:|Payment Terms:)",
            r"\n\1",
            raw_description
        ).strip()

        # Extract SKU (Module number value)
        sku_match = re.search(r"Module number[:：]?\s*([A-Za-z0-9\-/_.]+)", raw_description, re.IGNORECASE)
        sku = sku_match.group(1).strip() if sku_match else "N/A"

        # Add to list
        scraped_data.append({
            "Product Title": product_title,
            "Images": product_images,
            "Description": formatted_description,
            "Short description": formatted_description,
            "SKU": sku
        })

    except Exception as e:
        print(f"Error scraping {url}: {e}")
        scraped_data.append({
            "SKU": "ERROR",
            "Product Title": "ERROR",
            "Images": "ERROR",
            "Description": f"Failed to scrape: {url}",
            "Short description": f"Failed to scrape: {url}"
        })


df = pd.DataFrame(scraped_data)
df.to_csv(OUTPUT_FILE, index=False)
print(f"Scraping complete. Data saved to {OUTPUT_FILE}")


Scraping: https://www.yaotaishcnc.com/fanuc/fanuc-pcb-board-a16b-2202-0012.html
Scraping: https://www.yaotaishcnc.com/fanuc/fanuc-pcb-board-a16b-2202-0020.html
Scraping: https://www.yaotaishcnc.com/fanuc/fanuc-pcb-board-a16b-2202-0030.html
Scraping: https://www.yaotaishcnc.com/fanuc/fanuc-pcb-board-a16b-2202-0100.html
Scraping: https://www.yaotaishcnc.com/fanuc/fanuc-pcb-board-a16b-2202-0110.html
Scraping: https://www.yaotaishcnc.com/fanuc/fanuc-pcb-board-a16b-2202-0120.html
Scraping: https://www.yaotaishcnc.com/fanuc/fanuc-pcb-board-a16b-2202-0140.html
Scraping: https://www.yaotaishcnc.com/fanuc/fanuc-pcb-board-a16b-2202-0141.html
Scraping: https://www.yaotaishcnc.com/fanuc/fanuc-pcb-board-a16b-2202-0180.html
Scraping: https://www.yaotaishcnc.com/fanuc/fanuc-pcb-board-a16b-2202-0181.html
Scraping: https://www.yaotaishcnc.com/fanuc/fanuc-pcb-board-a16b-2202-0184.html
Scraping: https://www.yaotaishcnc.com/fanuc/fanuc-pcb-board-a16b-2202-0191.html
Scraping: https://www.yaotaishcnc.com/fa

In [26]:
# step 4

import pandas as pd
import os

# Example variable values — make sure these are defined before running the script
# __brand = "Fanuc"
# __start_page = 1
# _end_page = 5

df = pd.read_csv("./Output CSV/products.csv")

# Drop rows where any cell contains 'N/A'
# df_cleaned = df.replace("N/A", pd.NA).dropna()
df_cleaned = df

# Add 'Brand' column with all values set to '__brand'
df_cleaned["Brands"] = __brand

# Create the output folder if it doesn't exist
output_folder = "OutPut final"
os.makedirs(output_folder, exist_ok=True)

# Construct the filename using the variables
filename = f"{__brand} {__start_page} - {_end_page}.csv"
output_path = os.path.join(output_folder, filename)

# Save the cleaned DataFrame
df_cleaned.to_csv(output_path, index=False)

print(f"Cleaned file saved to: {output_path}")


Cleaned file saved to: OutPut final\Fanuc 461 - 600.csv


160 Pages in 16minutes Scraped of all links and images

In [ ]:
#step 4

# import pandas as pd
# import os

# df = pd.read_csv("./Output CSV\products.csv")

# # Drop rows where any cell contains 'N/A'
# # df_cleaned = df.replace("N/A", pd.NA).dropna()
# df_cleaned = df
# # Add 'Brand' column with all values set to 'Fanuc'
# df_cleaned["Brands"] = __brand

# # Create the output folder if it doesn't exist
# output_folder = "OutPut final"
# os.makedirs(output_folder, exist_ok=True)

# # Save the cleaned DataFrame
# output_path = os.path.join(output_folder, "cleaned_file.csv")
# df_cleaned.to_csv(output_path, index=False)

# print(f"Cleaned file saved to: {output_path}")